# Pandas data frame

Pandas è una libreria che ci permette di manipolare facilmente i dataframes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv('/home/mauro/python/family.csv')
print(df)

## Estrazione dati da pagine web

Una funzionalità molto interessante di Pandas è quella che ci permette di estrarre facilmente dati da pagine web con pochissime righe di codice.

Su Wikipedia troviamo l'elenco dei comuni italiani con più di 50 000 abitanti aggiornati al 31 dicembre 2021

N.B.
Occorre avere installato lxml, html5lib, bs4

In [ ]:
url_popolazione = "https://it.wikipedia.org/wiki/Comuni_d%27Italia_per_popolazione"
url_superficie = "https://it.wikipedia.org/wiki/Primi_100_comuni_italiani_per_superficie"

popolazione = pd.read_html(url_popolazione, attrs={"class":"wikitable"},header=0)[0]
popolazione.head(10)

Riordiniamo le città per regione e quindi per provincia

In [ ]:
popolazione = popolazione.sort_values(by=['Regione', 'Provincia / Città metropolitana'])
popolazione.head(10)

Sarebbe meglio riordinare le colonne mettendo al primo posto la regione e al secondo la provincia. La colonna N° non ci serve quindi la rimuoviamo.

Rinominiamo poi la colonna 'Provincia / Città metropolitana' semplicemente in "Provincia"

In [ ]:
cols = ['Regione', 'Provincia / Città metropolitana', 'Comune', 'Abitanti']
popolazione = popolazione[cols]

popolazione.rename(columns = {'Provincia / Città metropolitana':'Provincia'}, inplace = True)
popolazione.head(10)

Vediamo come sono stati importati i dati

### Contare il numero di comuni per ciascuna regione

In [ ]:
popolazione.groupby(['Regione']).size().reset_index(name='Comuni')

### Contare il numero di comuni per ciascuna Regione e provincia ordinandoli per numero di comuni

In [ ]:
popolazione.groupby(['Regione','Provincia']) \
    .size() \
    .reset_index(name='N° comuni') \
    .sort_values(by=['N° comuni'],ascending=False) \
    .head(10)

### Contare gli abitanti di ciascuna regione

Dobbiamo semplicemente applicare la funzione sum(), ma proviamo ad eseguire il prossimo frammento di codice

In [ ]:

popolazione.groupby(['Regione'])['Abitanti'].sum()

Il risultato non è quello sperato. Vengono infatti concatenate tutte le stringhe che rappresentano gli abitanti di ciascuna regione. Perchè questo?

Proviamo a vedere come è strutturato il nostro dataframe

In [ ]:
popolazione.dtypes

Il campo abitanti è di tipo object, quindi non possiamo usarlo per fare calcoli matematici. 

Bisogna convertire quell'oggetto in un numero intero.

ATTENZIONE qua c'e' un problema non banale. Nella stringa che mostra il numero degli abitanti lo spazio non è un semplice spazio ma è il carattere `xa0` che è un "non-breaking Latin1 (ISO 8859-1) space". Questo carattere va eliminato prima di poter procedere con la conversione.

Per approfondire: https://pbpython.com/pandas-html-table.html

In [ ]:
from unicodedata import normalize

def clean_normalize_whitespace(x):
    if isinstance(x, str):
        return normalize('NFKC', x).replace(' ','')
    else:
        return x

popolazione['Abitanti'] = popolazione['Abitanti'].map(clean_normalize_whitespace)
popolazione['Abitanti'] = pd.to_numeric(popolazione['Abitanti'])

abitanti_regione = popolazione.groupby(['Regione'])['Abitanti'] \
    .sum() \
    .reset_index() \
    .sort_values(by = ['Abitanti'], ascending=[False]) \

abitanti_regione.head(12)

In questo modo otteniamo il risultato voluto

# Grafici

Proviamo ora ad usare i dati appena calcolati per disegnare un grafico con il numero di abitanti per ciascuna regione

Barchart fa una colonna per ciascun dato
Istogram fa una colonna per ogni gruppo di dati (questi vanno prima raggruppati)
https://problemsolvingwithpython.com/06-Plotting-with-Matplotlib/06.06-Bar-Charts-and-Pie-Charts/#bar-charts
https://stackoverflow.com/questions/42421555/matplotlib-bar-chart-with-labels-space-out-the-bars

In [ ]:
ar = abitanti_regione.sort_values(by = ['Regione'], ascending=[False])

fig, ax = plt.subplots(figsize=(30,30))

labels = ar['Regione']
x_pos = np.arange(len(labels))
data = ar['Abitanti']

ax.bar(x_pos, data, align='center', alpha=0.5)

ax.set_ylabel('Abitanti', fontsize=50, rotation=90)

ax.set_xticks(x_pos)
ax.set_xticklabels(labels, fontsize=30, rotation=90)

ax.set_title('Numero di abitanti per regione', fontsize=50)

ax.yaxis.grid(True)

# Save the figure and show
plt.tight_layout()
plt.savefig('bar_plot.png')
plt.show()

# Esercizio

Scaricare le tabelle delle città per ciascuna regione dalla pagina
https://it.wikipedia.org/wiki/Titolo_di_citt%C3%A0_in_Italia



In [ ]:
url = 'https://it.wikipedia.org/wiki/Titolo_di_citt%C3%A0_in_Italia'
tabelle = pd.read_html(url, attrs={"class":"wikitable"},header=0)



In [ ]:
cs = pd.read_html(url_superficie, attrs={"class":"wikitable"},header=0)[0]
cs.head()